In [1]:
import pandas as pd

In [5]:
data = pd.read_csv(r"Datas//flights_merged_by_Hiral.csv")
data.head()

,sentiment,Reviews
0,negative,Never flying with them again! The horrible was...
1,positive,Such a smooth experience! The perfect was air ...
2,negative,A huge disappointment. The uncomfortable was f...
3,positive,A truly enjoyable flight! The amazing was seat...
4,neutral,Fairly standard. The average was baggage handl...


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8371 entries, 0 to 8370
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sentiment  8371 non-null   object
 1   Reviews    8371 non-null   object
dtypes: object(2)
memory usage: 130.9+ KB


In [8]:
df = pd.read_csv(r"Datas//flights_merged_by_Hiral.csv", usecols=["Reviews", "sentiment"])

In [9]:
# Check if any sentiment values exceed the allowed length
max_length = df["sentiment"].apply(len).max()
print(f"Max sentiment length after cleaning: {max_length}")

# Print rows where sentiment is longer than 20 characters
problematic_rows = df[df["sentiment"].apply(len) > 20]
print("Problematic rows:")
print(problematic_rows)


Max sentiment length after cleaning: 8
Problematic rows:
Empty DataFrame
Columns: [sentiment, Reviews]
Index: []


In [10]:
import re

df["sentiment"] = df["sentiment"].apply(lambda x: re.sub(r"\s+", " ", x).strip())  # Remove extra spaces


In [ ]:
import pandas as pd
import mysql.connector
import os 
from dotenv import load_dotenv
from sqlalchemy import create_engine

# Database connection details


load_dotenv()
DB_URL = os.getenv("DB_URL")
engine = create_engine(DB_URL)


# Read CSV file
csv_file_path = r"Datas//flights_merged_by_Hiral.csv"
df = pd.read_csv(csv_file_path, usecols=["Reviews", "sentiment"])

# Ensure sentiment values are strings, strip extra spaces, and within the correct length
df["sentiment"] = df["sentiment"].astype(str).str.strip()  # Remove leading/trailing spaces
df["sentiment"] = df["sentiment"].str[:20]  # Trim to 20 characters max

# Check if any sentiment values exceed 20 characters
print(f"Max sentiment length after cleaning: {df['sentiment'].astype(str).apply(len).max()}")

# Verify if any problematic rows exist
problematic_rows = df[df["sentiment"].apply(len) > 20]
print(f"Problematic rows: \n{problematic_rows}")
# Establish database connection
try:
    conn = mysql.connector.connect(**DB_CONFIG)
    cursor = conn.cursor()
    print("Database connection established.")

    print("First few rows before inserting:")
    print(df[:5])  # Show first 5 rows

    # SQL query to insert data
    sql = "INSERT INTO review (reviews, sentiment) VALUES (%s, %s)"

    # Define batch size
    batch_size = 100  
    data = list(df.itertuples(index=False, name=None))

    # Insert data in batches
    for i in range(0, len(data), batch_size):
        batch = data[i : i + batch_size]
        cursor.executemany(sql, batch)  
        conn.commit()
        print(f"Inserted {i + len(batch)} records...")  

    print("All records inserted successfully.")

except mysql.connector.Error as err:
    print(f"Error: {err}")

finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()
    print("Database connection closed.")


Host: None, User: None
Max sentiment length after cleaning: 8
Problematic rows: 
Empty DataFrame
Columns: [sentiment, Reviews]
Index: []


NameError: name 'cursor' is not defined

In [29]:
# import mysql.connector
# import pandas as pd

# # Load the CSV file into a DataFrame
# df = pd.read_csv(r"C:\Users\RUTVIK\flight_pipeline\Merged_df.csv")


# cursor = conn.cursor()

# ### Insert Data into review Table ###
# insert_review_query = """
# INSERT INTO review (reviews, sentiment) VALUES (%s, %s)
# """

# batch_size = 500  # Insert 500 rows at a time
# batch = []

# for i, row in df.iterrows():
#     batch.append((row["Reviews"], row["sentiment"]))
    
#     if len(batch) >= batch_size:
#         cursor.executemany(insert_review_query, batch)
#         conn.commit()  # Commit every 500 records
#         batch = []  # Reset batch

# # Insert any remaining records
# if batch:
#     cursor.executemany(insert_review_query, batch)
#     conn.commit()

# # import mysql.connector
# # import pandas as pd
# # import os

# # # Load the CSV file into a DataFrame
# # df = pd.read_csv(r"C:\Users\RUTVIK\flight_pipeline\Merged_df.csv")

# # # Establish MySQL database connection
# # host = os.getenv("DB_HOST")
# # user = os.getenv("DB_USER")
# # password = os.getenv("DB_PASSWORD")
# # database = os.getenv("flight")

# # # Print to check if values are loaded correctly
# # print(f"Host: {host}, User: {user}")

# # try:
# #     cursor = conn.cursor()

# #     # Insert Data into review Table
# #     insert_review_query = """
# #     INSERT INTO review (reviews, sentiment) VALUES (%s, %s)
# #     """

# #     batch_size = 500  # Insert 500 rows at a time
# #     batch = []

# #     for i, row in df.iterrows():
# #         batch.append((row["Reviews"], row["sentiment"]))
        
# #         if len(batch) >= batch_size:
# #             cursor.executemany(insert_review_query, batch)
# #             conn.commit()  # Commit every 500 records
# #             batch = []  # Reset batch

# #     # Insert any remaining records
# #     if batch:
# #         cursor.executemany(insert_review_query, batch)
# #         conn.commit()
        
# #     print("Reviews successfully imported into the review table!")
    
# # finally:
# #     # Close the cursor and connection
# #     cursor.close()
# #     conn.close()

    



In [30]:
import mysql.connector
import pandas as pd
import os 

# Fetch database credentials
host = os.getenv("DB_HOST")
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
database = os.getenv("flight")

# Print to check if values are loaded correctly
print(f"Host: {host}, User: {user}")

# Function to fetch data from MySQL
def fetch_data_from_db(table_name):
    try:
        # Establish connection
        conn = mysql.connector.connect(**DB_CONFIG)
        
        # Query to select all data from the given table
        query = f"SELECT * FROM {table_name}"
        
        # Load data into Pandas DataFrame
        df = pd.read_sql(query, conn)
        
        # Close connection
        conn.close()
        
        print("Data retrieved successfully!")
        return df

    except mysql.connector.Error as e:
        print(f"Error fetching data: {e}")
        return None

# Specify the table name
table_name = "review"

# Retrieve data and store in DataFrame
df = fetch_data_from_db(table_name)

# Display first few rows
if df is not None:
    print(df.head())


Host: localhost, User: root


C:\Users\RUTVIK\AppData\Local\Temp\ipykernel_19964\2679510178.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Data retrieved successfully!
                                             reviews sentiment
0  Never flying with them again! The horrible was...  positive
1  Never flying with them again! The horrible was...  negative
2  Such a smooth experience! The perfect was air ...  positive
3  A huge disappointment. The uncomfortable was f...  negative
4  A truly enjoyable flight! The amazing was seat...  positive


In [5]:
!pip install joblib


   ---------------------------------------- 0.0/301.8 kB ? eta -:--:--
   ---------------------------------------- 0.0/301.8 kB ? eta -:--:--
   ---------------------------------------- 0.0/301.8 kB ? eta -:--:--
   ---------------------------------------- 0.0/301.8 kB ? eta -:--:--
   - -------------------------------------- 10.2/301.8 kB ? eta -:--:--
   ----- --------------------------------- 41.0/301.8 kB 393.8 kB/s eta 0:00:01
   --------- ----------------------------- 71.7/301.8 kB 491.5 kB/s eta 0:00:01
   ------------------ ------------------- 143.4/301.8 kB 774.0 kB/s eta 0:00:01
   ---------------------------------------  297.0/301.8 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 301.8/301.8 kB 1.2 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
!pip install sqlalchemy


   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
    --------------------------------------- 0.0/2.1 MB 435.7 kB/s eta 0:00:05
   - -------------------------------------- 0.1/2.1 MB 544.7 kB/s eta 0:00:04
   --- ------------------------------------ 0.2/2.1 MB 1.2 MB/s eta 0:00:02
   -------- ------------------------------- 0.4/2.1 MB 2.1 MB/s eta 0:00:01
   ------------- -------------------------- 0.7/2.1 MB 2.8 MB/s eta 0:00:01
   ------------------ --------------------- 1.0/2.1 MB 3.3 MB/s eta 0:00:01
   ----------------------- ---------------- 1.2/2.1 MB 3.5 MB/s eta 0:00:01
   --------------------------- ------------ 1.5/2.1 MB 3.7 MB/s eta 0:00:01
   --------------------------------- ------ 1.8/2.1 MB 4.0 MB/s eta 0:00:01
   ---------------------------------------  2.1/2.1 MB 4.2 MB/s eta 0:00:01
   --------------------------


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
!pip install scikit-learn


     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ------------------- ------------------ 30.7/60.8 kB 660.6 kB/s eta 0:00:01
     -------------------------------------- 60.8/60.8 kB 647.7 kB/s eta 0:00:00
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 0.1/11.1 MB 3.6 MB/s eta 0:00:04
   - -------------------------------------- 0.3/11.1 MB 3.8 MB/s eta 0:00:03
   -- ------------------------------------- 0.7/11.1 MB 4.6 MB/s eta 0:00:03
   --- ------------------------------------ 0.9/11.1 MB 4.7 MB/s eta 0:00:03
   --- ------------------------------------ 1.1/11.1 MB 5.0 MB/s eta 0:00:02
   ----- ---------------------------------- 1.5/11.1 MB 5.3 MB/s eta 0:00:02
   ------ --------------------------------- 1.9/11.1 MB 5.7 MB/s eta 0:00:02
   ------- -------------------------------- 2.2/11.1 MB 5.8 MB/s eta 0:00:02
   -------


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
!pip install xgboost

  Using cached xgboost-2.1.4-py3-none-win_amd64.whl.metadata (2.1 kB)
Using cached xgboost-2.1.4-py3-none-win_amd64.whl (124.9 MB)



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
!pip install pymysql


   ---------------------------------------- 0.0/45.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/45.0 kB ? eta -:--:--
   --------- ------------------------------ 10.2/45.0 kB ? eta -:--:--
   ------------------ --------------------- 20.5/45.0 kB 131.3 kB/s eta 0:00:01
   ---------------------------------------- 45.0/45.0 kB 318.6 kB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve DB URL from environment variables
DB_URL = os.getenv("DB_URL")

# Check if the value is loaded
if DB_URL:
    print("DB_URL loaded successfully!")
else:
    print("Failed to load DB_URL. Check your .env file.")

DB_URL loaded successfully!


In [4]:
import os
import re
import numpy as np
import pandas as pd
import joblib
import pickle
import mysql.connector
from sqlalchemy import create_engine
from dotenv import load_dotenv
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load environment variables
load_dotenv()
DB_URL = os.getenv("DB_URL")

# Establish database connection
engine = create_engine(DB_URL)

# Load flight reviews from database
with engine.connect() as conn:
    df = pd.read_sql('SELECT * FROM review', conn)

# Text Cleaning Class
class TextCleaner(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X.apply(self.clean_text)
    
    @staticmethod
    def clean_text(text):
        text = str(text).lower()
        text = re.sub(r'\W', ' ', text)
        text = re.sub(r'\s+', ' ', text).strip()
        return text

# Apply text cleaning
cleaner = TextCleaner()
df['cleaned_review'] = cleaner.transform(df['reviews'])

# Encode sentiment labels
sentiment_mapping = {"positive": 1, "negative": 0, "neutral": 2}
df["sentiment"] = df["sentiment"].map(sentiment_mapping)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(
    df["cleaned_review"], df["sentiment"], test_size=0.2, random_state=42, stratify=df["sentiment"]
)

# TF-IDF Vectorizer
vectorizer = TfidfVectorizer(max_features=50000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Save vectorizer
pickle.dump(vectorizer, open("tfidf_vectorizer.pkl", "wb"))

# Train Random Forest Model
rf_model = RandomForestClassifier(n_estimators=200, max_depth=30, random_state=42)
rf_model.fit(X_train_tfidf, y_train)
joblib.dump(rf_model, "random_forest.pkl")

# Train XGBoost Model
xgb_model = xgb.XGBClassifier(n_estimators=300, learning_rate=0.1, max_depth=6, eval_metric="mlogloss")
xgb_model.fit(X_train_tfidf, y_train)
joblib.dump(xgb_model, "xgboost.pkl")

# Train Deep Learning Model
tokenizer = Tokenizer(num_words=50000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)
X_train_seq = pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen=100, padding="post")
X_test_seq = pad_sequences(tokenizer.texts_to_sequences(X_test), maxlen=100, padding="post")

model = Sequential([
    Embedding(input_dim=50000, output_dim=32, input_length=100),
    GlobalAveragePooling1D(),
    Dense(64, activation="relu"),
    Dense(3, activation="softmax")
])

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit(X_train_seq, y_train, epochs=10, batch_size=256, validation_data=(X_test_seq, y_test))
model.save("tf_sentiment_model.h5")
pickle.dump(tokenizer, open("tokenizer.pkl", "wb"))

# Evaluate Random Forest Model
rf_accuracy = accuracy_score(y_test, rf_model.predict(X_test_tfidf))
print(f"Random Forest Accuracy: {rf_accuracy:.4f}")

# Evaluate XGBoost Model
xgb_accuracy = accuracy_score(y_test, xgb_model.predict(X_test_tfidf))
print(f"XGBoost Accuracy: {xgb_accuracy:.4f}")

# Evaluate Deep Learning Model
dl_accuracy = model.evaluate(X_test_seq, y_test, verbose=0)[1]
print(f"Deep Learning Model Accuracy: {dl_accuracy:.4f}")

# Determine the best model
best_model_name = "Random Forest" if rf_accuracy >= xgb_accuracy and rf_accuracy >= dl_accuracy else (
    "XGBoost" if xgb_accuracy >= dl_accuracy else "Deep Learning"
)
print(f"Best Model: {best_model_name}")

# Continue with the further process using the best model
best_model = rf_model if best_model_name == "Random Forest" else (
    xgb_model if best_model_name == "XGBoost" else model
)

# Sentiment Pipeline
vectorizer = pickle.load(open("tfidf_vectorizer.pkl", "rb"))

if best_model_name == "Deep Learning":
    sentiment_pipeline = Pipeline([
        ('text_cleaning', TextCleaner()),
        ('tokenizer', Tokenizer(num_words=50000, oov_token="<OOV>")),
        ('pad_sequences', None),  # Custom transformer for padding sequences can be created
        ('classifier', best_model)
    ])
else:
    sentiment_pipeline = Pipeline([
        ('text_cleaning', TextCleaner()),
        ('vectorizer', vectorizer),
        ('classifier', best_model)
    ])

joblib.dump(sentiment_pipeline, "sentiment.pkl")

# Sentiment Prediction Function
def predict_sentiment(review_text):
    review_df = pd.DataFrame({"review": [review_text]})
    prediction = sentiment_pipeline.predict(review_df["review"])[0]
    sentiment_labels = {1: "Positive", 0: "Negative", 2: "Neutral"}
    return sentiment_labels[prediction]

# Load flight reviews for inference
with engine.connect() as conn:
    inference_reviews = pd.read_sql('SELECT * FROM review', conn)

inference_reviews["Predicted_Sentiment"] = inference_reviews["reviews"].apply(lambda x: predict_sentiment(x))

print(inference_reviews)




Epoch 1/10


c:\Users\RUTVIK\flight_pipeline\flight\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


367/367 ━━━━━━━━━━━━━━━━━━━━ 13s 27ms/step - accuracy: 0.7729 - loss: 0.7041 - val_accuracy: 1.0000 - val_loss: 0.0088
Epoch 2/10
367/367 ━━━━━━━━━━━━━━━━━━━━ 9s 23ms/step - accuracy: 1.0000 - loss: 0.0057 - val_accuracy: 1.0000 - val_loss: 0.0012
Epoch 3/10
367/367 ━━━━━━━━━━━━━━━━━━━━ 9s 24ms/step - accuracy: 1.0000 - loss: 9.8884e-04 - val_accuracy: 1.0000 - val_loss: 4.0296e-04
Epoch 4/10
367/367 ━━━━━━━━━━━━━━━━━━━━ 8s 22ms/step - accuracy: 1.0000 - loss: 3.7895e-04 - val_accuracy: 1.0000 - val_loss: 2.0822e-04
Epoch 5/10
367/367 ━━━━━━━━━━━━━━━━━━━━ 8s 21ms/step - accuracy: 1.0000 - loss: 3.0668e-04 - val_accuracy: 1.0000 - val_loss: 1.2431e-04
Epoch 6/10
367/367 ━━━━━━━━━━━━━━━━━━━━ 7s 20ms/step - accuracy: 1.0000 - loss: 2.4989e-04 - val_accuracy: 1.0000 - val_loss: 8.2188e-05
Epoch 7/10
367/367 ━━━━━━━━━━━━━━━━━━━━ 7s 20ms/step - accuracy: 1.0000 - loss: 8.9634e-05 - val_accuracy: 1.0000 - val_loss: 6.8680e-05
Epoch 8/10
367/367 ━━━━━━━━━━━━━━━━━━━━ 7s 20ms/step - accuracy: 1.

Random Forest Accuracy: 1.0000
XGBoost Accuracy: 1.0000
Deep Learning Model Accuracy: 1.0000
Best Model: Random Forest
                                                  reviews sentiment  \
0       Never flying with them again! The horrible was...  positive   
1       Never flying with them again! The horrible was...  negative   
2       Such a smooth experience! The perfect was air ...  positive   
3       A huge disappointment. The uncomfortable was f...  negative   
4       A truly enjoyable flight! The amazing was seat...  positive   
...                                                   ...       ...   
117190  Never flying with them again! The frustrating ...  negative   
117191  Not the best, not the worst—just fair. The WiF...   neutral   
117192  Pretty average overall. The reasonable was bag...   neutral   
117193  I had an amazing flight. The amazing was bagga...  positive   
117194  I can't believe how horrible this flight was. ...  negative   

       Predicted_Sentiment  

In [9]:
host = os.getenv("DB_HOST")
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
database = os.getenv("flight")

# Print to check if values are loaded correctly
print(f"Host: {host}, User: {user}")

Host: localhost, User: root


In [ ]:
# Read CSV file
csv_file_path = r"C:\Users\RUTVIK\flight_pipeline\Merged_df.csv"
df = pd.read_csv(csv_file_path, usecols=["Reviews"])

try:
    conn = mysql.connector.connect(**DB_CONFIG)
    cursor = conn.cursor()
    print("Database connection established.")

    # print("First few rows before inserting:")
    # print(df[:5])  # Show first 5 rows

    # SQL query to insert data
    sql = "INSERT INTO Test_review (Reviews) VALUES (%s)"

    # Define batch size
    batch_size = 100  
    data = list(df.itertuples(index=False, name=None))

    # Insert data in batches
    for i in range(0, len(data), batch_size):
        batch = data[i : i + batch_size]
        cursor.executemany(sql, batch)  
        conn.commit()
        print(f"Inserted {i + len(batch)} records...")  

    print("All records inserted successfully.")

except mysql.connector.Error as err:
    print(f"Error: {err}")

finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()
    print("Database connection closed.")

Database connection established.
Inserted 100 records...
Inserted 200 records...
Inserted 300 records...
Inserted 400 records...
Inserted 500 records...
Inserted 600 records...
Inserted 700 records...
Inserted 800 records...
Inserted 900 records...
Inserted 1000 records...
Inserted 1100 records...
Inserted 1200 records...
Inserted 1300 records...
Inserted 1400 records...
Inserted 1500 records...
Inserted 1600 records...
Inserted 1700 records...
Inserted 1800 records...
Inserted 1900 records...
Inserted 2000 records...
Inserted 2100 records...
Inserted 2200 records...
Inserted 2300 records...
Inserted 2400 records...
Inserted 2500 records...
Inserted 2600 records...
Inserted 2700 records...
Inserted 2800 records...
Inserted 2900 records...
Inserted 3000 records...
Inserted 3100 records...
Inserted 3200 records...
Inserted 3300 records...
Inserted 3400 records...
Inserted 3500 records...
Inserted 3600 records...
Inserted 3700 records...
Inserted 3800 records...
Inserted 3900 records...
I

In [8]:
import joblib
import numpy as np
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import pandas as pd
# from CarReviews_model_pipeline import TextCleaner

# Load the trained model
sentiment = joblib.load("sentiment.pkl")
# print(sentiment)

# Load environment variables
load_dotenv()
DB_URL = os.getenv("DB_URL")

# Establish database connection
engine = create_engine(DB_URL)

with engine.connect() as conn:
    inference_Review_car = pd.read_sql('SELECT * FROM Test_review', conn)


def predict_sentiment(review_text):
    review_df = pd.DataFrame({"review": [review_text]})  # Convert to DataFrame
    prediction = sentiment.predict(review_df["review"])[0]  # Predict
    # sentiment_labels = {1: "Positive", 0: "Negative", 2: "Neutral"}
    return prediction

inference_Review_car["Predicted_Sentiment"] = inference_Review_car["reviews"].apply(lambda x: predict_sentiment(x))

print(inference_Review_car)

                                                reviews  Predicted_Sentiment
0     Never flying with them again! The horrible was...                    0
1     Such a smooth experience! The perfect was air ...                    1
2     A huge disappointment. The uncomfortable was f...                    0
3     A truly enjoyable flight! The amazing was seat...                    1
4     Fairly standard. The average was baggage handl...                    2
...                                                 ...                  ...
8366  Never flying with them again! The frustrating ...                    0
8367  Not the best, not the worst—just fair. The WiF...                    2
8368  Pretty average overall. The reasonable was bag...                    2
8369  I had an amazing flight. The amazing was bagga...                    1
8370  I can't believe how horrible this flight was. ...                    0

[8371 rows x 2 columns]
